# Day 5

In [1]:
from aocd.models import Puzzle
from pathlib import Path
puzzle = Puzzle(year=2025, day=int(Path(__vsc_ipynb_file__).stem))
puzzle.url

'https://adventofcode.com/2025/day/5'

In [5]:
example = """3-5
10-14
16-20
12-18

1
5
8
11
17
32"""

In [21]:
def parse_fresh_ingredient_range(line: str) -> tuple[int,int]:
    start, end = line.strip().split('-')
    sn = int(start)
    en = int(end)
    assert(sn <= en)
    return (sn, en)

assert(parse_fresh_ingredient_range('3-5') == (3,5))

In [22]:
def parse_fresh_ingredient_ranges(lines: str) -> list[tuple[int,int]]:
    lines_list = lines.strip().split()
    return [parse_fresh_ingredient_range(line) for line in lines_list]

assert(parse_fresh_ingredient_ranges('3-5\n100-150') == [(3,5), (100,150)])

In [24]:
def parse_ingredient(line: str)-> int:
    return int(line.strip())

assert(parse_ingredient('2') == 2)

In [25]:
def parse_ingredients(lines:str)-> list[int]:
    return [parse_ingredient(line) for line in lines.strip().split()]

assert(parse_ingredients('2\n3') == [2,3])

In [27]:
class Interval:
    def __init__(self, begin: int, end: int, data=None):
        self.begin = begin
        self.end = end
        self.data = data

    def __repr__(self):
        return f"Interval({self.begin}, {self.end}, {self.data})"

    def overlaps(self, other_interval: Interval):
        return max(self.begin, other_interval.begin) < min(self.end, other_interval.end)

class IntervalNode:
    def __init__(self, interval: Interval):
        self.interval = interval
        self.max_end = interval.end
        self.left: IntervalNode | None = None
        self.right: IntervalNode | None = None

class IntervalTree:
    def __init__(self):
        self.root = None

    def insert(self, interval: Interval):
        self.root = self._insert(self.root, interval)

    def _insert(self, node: IntervalNode | None, interval: Interval):
        if node is None:
            return IntervalNode(interval)

        # Update max_end
        node.max_end = max(node.max_end, interval.end)

        if interval.begin < node.interval.begin:
            node.left = self._insert(node.left, interval)
        else:
            node.right = self._insert(node.right, interval)
        return node

    def query(self, query_interval: Interval) -> list[Interval]:
        results: list[Interval] = []
        self._query(self.root, query_interval, results)
        return results

    def _query(self, node: IntervalNode | None, query_interval: Interval, results: list[Interval]):
        if node is None:
            return

        # If the current node's interval overlaps with the query interval, add it to results
        if node.interval.overlaps(query_interval):
            results.append(node.interval)

        # If the left child exists and its max_end is greater than or equal to the query_interval's begin,
        # there might be overlaps in the left subtree.
        if node.left is not None and node.left.max_end >= query_interval.begin:
            self._query(node.left, query_interval, results)

        # If the right child exists and the query_interval's end is greater than or equal to the current node's begin,
        # there might be overlaps in the right subtree.
        if node.right is not None and query_interval.end >= node.interval.begin:
            self._query(node.right, query_interval, results)


In [37]:
def tee(s: int) -> int:
    print(s)
    return s

In [38]:
def solve_a(input: str) -> int:
    a, b = input.strip().split('\n\n')
    fresh_ranges, ingredients = parse_fresh_ingredient_ranges(a), parse_ingredients(b)
    fresh_tree = IntervalTree()
    for begin,end in fresh_ranges:
        fresh_tree.insert(Interval(begin, end+1))
    
    count = 0
    for ingredient in ingredients:
        ingredient_interval = Interval(ingredient,ingredient+1)
        result = fresh_tree.query(ingredient_interval)
        if result:
            count += 1
    return count
    
assert(solve_a(example) == 3)
puzzle.answer_a = tee(solve_a(puzzle.input_data))

643


In [39]:
def solve_b(input: str) -> int:
    a = input.strip().split('\n\n')[0]
    intervals = parse_fresh_ingredient_ranges(a)

    intervals.sort(key=lambda x: x[0])

    total_area = 0
    first_interval = intervals[0]
    cmi_start, cmi_end = first_interval[0], first_interval[1]+1

    for i_start, i_end in intervals[1:]:
        i_end += 1
        if i_start <= cmi_end:
            # Overlap or touch, merge
            cmi_end = max(cmi_end, i_end)
        else:
            # No overlap, add current merged interval and start a new one
            total_area += cmi_end - cmi_start
            cmi_start, cmi_end = i_start, i_end
    total_area += cmi_end - cmi_start # Add the last merged interval
    return total_area
   
assert(solve_b(example) == 14)
puzzle.answer_b = tee(solve_b(puzzle.input_data))

342018167474526
